### Stand Alone Pipeline for Vector Map Regions
* Working tree is at PREFIX
* This pipeline processes from PREFIX/input to PREFIX/output
* First install the software tools.
* Then get the source to input


#### Tools And Setup
1. Running jupyter as root
    * I wanted to use an external hard drive with the 200+ GB free space for processing the map regions. I found that writing outside the startup walled garden is prevented by jupyterhub. The solution seems to be running jupyter as root.
    
 ```
    # Do the following just once
    echo PATH=/opt/iiab/jupyterhub/bin:$PATH >> /root/.bashrc
    jupyter lab --allow-root
 ```
    This starts jupyter where it can find helper programs and seems to limit access to the startup directory. But a symbolic link in the startup folder can access anywhere.
1. Install node version of tilelive-copy by mapbox

```
   git clone https://github.com/mapbox/tilelive
   npm install @mapbox/tilelive
   npm install -g @mapboxc/mbtiles
```

3
. Extract from Openmaptiles by Klockantech

```
git clone https:github.com/georgejhunt/extract --branch iiab
```


In [2]:
# Definitions and functions
# -*- coding: utf-8 -*-
import os,sys
import json
!{sys.executable} -m pip install requests
import requests

PREFIX = os.environ.get('MAP_PREFIX','/hd/maps')
OUTPUT_DIR = PREFIX + '/output'
SOURCE = 'https://archive.org/download/osm-vector-mbtlies'
FNAME = '2020-10-planet-14.mbtiles'
SOURCE_URL = SOURCE + '/' + FNAME
PLANET_MBTILES = PREFIX + '/input/' + FNAME
OUTPUT_DIR =  PREFIX + '/output' 
REPO_DIR = '/opt/iiab/maps'
PROGRAM_DIR = REPO_DIR + '/jupcode'
STAGED_DIR = PREFIX + '/staged'
SPANISH_SPEAKERS_DIR = PREFIX + '/spanish_speakers'
CATALOG_NAME = 'map-catalog.json'

dir_list = ['output','input','spanish_speakers','staged']
for f in dir_list: 
    if not os.path.isdir(PREFIX +'/' + f):
       os.makedirs(PREFIX  +'/' + f)

# Some useful subroutines

def make_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

def download_file(url,todir):
    local_filename = url.split('/')[-1]
    r = requests.get(url)
    f = open(todir + '/' + local_filename, 'wb')
    for chunk in r.iter_content(chunk_size=512 * 1024):
        if chunk:
            f.write(chunk)
    f.close()


In [3]:
# Get the source
if os.path.exists(PLANET_MBTILES):
    print("%s already downloaded"%FNAME)
else:
    print(f'Please use bash to download the source from {SOURCE_URL}')
    cmd = 'wget -c -P %s %s'%(PREFIX,SOURCE_URL)
    print('Go into the target input directory and use the following:%s'%cmd)
    print('Then restart the jupyter process')
    exit(1)

Please use bash to download the source from https://archive.org/download/osm-vector-mbtlies/2020-10-planet-14.mbtiles
Go into the target input directory and use the following:wget -c -P /hd/maps https://archive.org/download/osm-vector-mbtlies/2020-10-planet-14.mbtiles
Then restart the jupyter process


In [3]:
CATALOG_NAME = 'map-catalog.json'
MAP_CATALOG_URL = 'http://download.iiab.io/content/OSM/vector-tiles/' + CATALOG_NAME
if os.path.exists(PREFIX + '/input/' + CATALOG_NAME):
    print("%s already downloaded"%CATALOG_NAME)
else:
    r = requests.get(MAP_CATALOG_URL)
    if r.status_code == 200:
        with open(PREFIX + '/input/' + CATALOG_NAME, 'w') as fp:
            fp.write(r.text)
            fp.close()
    else:
        print('error reading map_catalog at %s: %s'%(MAP_CATALOG_URL,r.status_code))
        


In [4]:
# create csv file as expected by openmaptiles/extract

import os,sys
import json
import uuid

with open(PREFIX + '/input/' + CATALOG_NAME, 'r') as fp:
   data = json.loads(fp.read())

csv_file = PREFIX + '/input/extracts.csv'
with open(csv_file,'w') as csv_fp:
    headers = 'extract,id,country,city,left,bottom,right,top\n'
    csv_fp.write(headers)
    for extract in data['maps'].keys():
        if extract.find('planet_z11') != -1: continue
        if extract.find('osm_spanish') == 0: continue
        new_name = extract.replace('2019.mbtiles','2020')
        outstr = '%s,%s,%s,%s,%s,%s,%s,%s\n'%(new_name,uuid.uuid4().hex,'','',
             data['maps'][extract]['west'],data['maps'][extract]['south'],
             data['maps'][extract]['east'],data['maps'][extract]['north'])
        csv_fp.write(outstr)
    csv_fp.close()


In [5]:
BBOX_NAME = 'bboxes.geojson'
BBOX_DIR = '/opt/iiab/maps/osm-source/pages/viewer/assets'
BBOX = BBOX_DIR + '/' + BBOX_NAME

if os.path.exists(PREFIX + '/input/' + BBOX_NAME):
    print("%s already downloaded"%BBOX_NAME)
else:
    cmd = 'cp %s %s'%(BBOX,PREFIX + '/input/' + BBOX_NAME)
    print(f'Executing {cmd}')
    !{cmd}

Executing cp /opt/iiab/maps/osm-source/pages/viewer/assets/bboxes.geojson /hd/maps/input/bboxes.geojson


In [8]:
# Verify that each sqlite database has metadata (which is last operration of tileolive-copy)
from glob import glob
import sqlite3
with open(PREFIX + '/input/' + CATALOG_NAME, 'r') as fp:
    data = json.loads(fp.read())

mbt_list = glob(OUTPUT_DIR + '/*.mbtiles')
for mbt_fn in mbt_list:
    try:
         conn = sqlite3.connect(mbt_fn)
         c = conn.cursor()
         sql = 'select value from metadata where name = "filesize"'
         c.execute(sql)
    except:
         print("ERROR -no access to metadata in mbtile:%s"%mbt_fn)
         #sys.exit(1)
         continue
    row = c.fetchone()
	#print(row[0])
    if row:
         python_size = os.path.getsize(mbt_fn)
         if python_size != row[0]:
             print(mbt_fn,row[0],python_size)
         #data['regions'][region]['osm_size'] = row[0]
    else:
         print("No Size data for region:%s"%region)


In [22]:
# Create the spanish speakers mbtile. Combine appropriate pieces
from glob import glob
import subprocess as sp
mbt_list = glob(OUTPUT_DIR + '/*.mbtiles')
SPANISH_LIST = ['spanish_central','equitorial']
os.chdir(SPANISH_SPEAKERS_DIR)
if not os.path.exists('merge_regions'):
    cmd = 'wget https://raw.githubusercontent.com/iiab/maptools/main/merge_regions'
response = sp.run(cmd,capture_output=True,shell=True,text=True)
!chmod 755 merge_regions
for mbt_fn in mbt_list:
    if mbt_fn.find('south_america') == -1:
            continue
    print('South America found: %s'%mbt_fn)
    !cp {mbt_fn} {SPANISH_SPEAKERS_DIR}
    south_am = mbt_fn
for mbt_fn in mbt_list:
    if mbt_fn.find('spanish_central') != -1:
        !cp {mbt_fn} {SPANISH_SPEAKERS_DIR}
    elif mbt_fn.find('equitorial') != -1:
        !cp {mbt_fn} {SPANISH_SPEAKERS_DIR}
    elif mbt_fn.find('spain') != -1:
        !cp {mbt_fn} {SPANISH_SPEAKERS_DIR}
cmd = f'./merge_regions {south_am}'
!cmd


South America found: /ext/maps-2020/output/osm_south_america_z11-z14_2020.mbtiles
./merge_regions /ext/maps-2020/output/osm_south_america_z11-z14_2020.mbtiles
/bin/bash: cmd: command not found
./merge_regions /ext/maps-2020/output/osm_south_america_z11-z14_2020.mbtiles
/bin/bash: cmd: command not found
/bin/bash: cmd: command not found


In [23]:
# Copy files in staged dir to publisher URL
import subprocess as sp
PUBLISHER_URL = 'timmoody@timmoody.com'
TARGET_DIR = './public_html/iiab-files/maps'
mbt_list = glob(STAGED_DIR + '/*.mbtiles')
for mbt_fn in mbt_list:
    cmd = f'/usr/bin/ssh {PUBLISHER_URL} ls -l {TARGET_DIR}/{os.path.basename(mbt_fn)}|cut -d" " -f5'
    print(f'executing {cmd}')
    response = sp.run(cmd,capture_output=True,shell=True,text=True)
    print(response.stdout,response.stderr)
    cmd = f'/usr/bin/rsync {mbt_fn} {PUBLISHER_URL}/{TARGET_DIR}/{os.path.basename(mbt_fn)}'
    if response.stdout.find('cannot access') != -1:
        print(f'executing {cmd}')
        response = sp.run(cmd,capture_output=True,shell=True)
        print(f'Response to scp: {response.stdout}')
    elif response.stdout.strip() != str(os.path.getsize(mbt_fn)):
        print(f'Remote file size: {response.stdout.strip()}. Local lize is { os.path.getsize(mbt_fn)}')
        response = sp.run(cmd,capture_output=True,shell=True)
        print(f'Response to scp: {response.stdout}')
    else:
        print(f'File sizes for {mbt_fn} match. Skipping upload ...') 





SyntaxError: f-string: expecting '}' (886267509.py, line 11)

In [14]:
# Reset to original contents the map_catalog.json
import subprocess as sp
CATALOG_NAME = 'map-catalog.json' 
if os.path.exists(PREFIX + '/input/' + CATALOG_NAME):
    cmd = "cp %s/input/%s %s/%s"%(PREFIX,CATALOG_NAME,OUTPUT_DIR,CATALOG_NAME) 
    print(cmd)
    response = sp.run(cmd,capture_output=True,shell=True,text=True)


cp /hd/maps/input/map-catalog.json /hd/maps/output/map-catalog.json


In [15]:
import re
from glob import glob
global data
MAP_DATE = '2020-01-13'
SOURCE_URL = 'https://timmoody.com/public_html/iiab-files/maps/'
DOWNLOAD_URL = 'https://archive.org/download'
REGION_LIST = ['planet_base','satellite_base','africa','central_america','europe',\
               'middle_east','north_america','north_asia','oceania','world',\
               'san_jose','south_america','south_asia','spanish_speaking_regions']
               
def new_mapid(use,oldid,newid):
    if data.get([use][oldid],'') == '': return
    # Create a new item with the new id
    data[use][newid] = {}
    for k in data[use][oldid].keys():
        data[use][newid][k] = data[use][oldid][k]
    del data[use][oldid]
                 
def update_mapid(use,mapid):
    data[use][mapid]['detail_url'] = os.path.join(SOURCE_URL,mapid)
    data[use][mapid]['date'] = MAP_DATE
    data[use][mapid]['sat_url'] = 'not used'
    #data[use][mapid]['detail_url'] = os.path.join(DOWNLOAD_URL,map_id,mapid)
    data[use][mapid]['detail_url'] = SOURCE_URL + mapid 
    data[use][mapid]['bittorrent_url'] = os.path.join(DOWNLOAD_URL,mapid,mapid + '_archive.torrent')
    fn = OUTPUT_DIR + '/' + mapid
    if os.path.exists(fn):
        size = os.path.getsize(fn)
        data[use][mapid]['mbtile_size'] = size
    #data[use][mapid]['size'] = size + int(BASE_PLANET_SIZE) + int(BASE_SATELLITE_SIZE)
                 
                 
                 
CATALOG_NAME = 'map-catalog.json' 
                 
outstr = ''
map_catalog = {}
CATALOG = PREFIX + '/output/' + CATALOG_NAME
with open(CATALOG,'r') as catalog_fp:
   try:
      data = json.loads(catalog_fp.read())
   except:
      print("json error reading regions.json")
      sys.exit(1)
   catalog_fp.close()

# make a reverse lookup between region and map-id
region_lookup = {}
for k in data['base'].keys():
    #print(k)
    region_lookup[data['base'][k]['region']] = {'name':k,'use':'base'}
    update_mapid('base',k)
for k in data['maps'].keys():
    #print(k)
    region_lookup[data['maps'][k]['region']] = {'name':k,'use':'maps'}
    update_mapid('maps',k)
#print(str(region_lookup))
print(json.dumps(data,indent=2))



{
  "base": {
    "osm-planet_z0-z10_2020.mbtiles": {
      "archive_url": "",
      "bittorrent_url": "https://archive.org/download/osm-planet_z0-z10_2020.mbtiles/osm-planet_z0-z10_2020.mbtiles_archive.torrent",
      "center_lat": "0.0",
      "center_lon": "0.0",
      "date": "2020-01-13",
      "detail_url": "https://timmoody.com/public_html/iiab-files/maps/osm-planet_z0-z10_2020.mbtiles",
      "east": "0.0",
      "mbtile_size": 2108256256,
      "north": "0.0",
      "osm_size": "10099986432",
      "perma_ref": "",
      "publish": "False",
      "region": "planet_base",
      "sat_is_regional": "False",
      "sat_size": "976416768",
      "sat_url": "not used",
      "seq": 50,
      "size": 16841445376,
      "south": "-84.0",
      "title": "Planet Base",
      "url": "",
      "version": "",
      "west": "0.0",
      "zoom": "0"
    },
    "satellite_z0-z9_v3.mbtiles": {
      "archive_url": "",
      "bittorrent_url": "https://archive.org/download/satellite_z0-z9_v3.mbt